## Ctrl-G Tutorial

### **Part B**. Ctrl-G on TULU2-7B (more computation required)

Step 1. load pretrained models.

In [2]:
!pip3 install protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 KB 8.7 MB/s eta 0:00:00a 0:00:01


In [1]:
%load_ext autoreload
%autoreload 2
    
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # set your cuda device
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import torch
import ctrlg
from transformers import AutoModelForCausalLM, AutoTokenizer, LogitsProcessorList

device = 'cuda'


# load the pretrained base_model and hmm_model;
BASE_MODEL_PATH = f'ctrlg/tulu2-7b_writing-prompts'
HMM_MODEL_PATH = f'ctrlg/hmm_tulu2-7b_writing-prompts_32768'

base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_PATH).to(device)
base_model.eval()
base_model.half() # fp16 inference
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH)
hmm_model = ctrlg.HMM.from_pretrained(HMM_MODEL_PATH).to(device)

/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Step 2. specify logical constraints as DFAs.

In [2]:
import pandas as pd
data = pd.read_csv('/media/data/bazaluk/ctrlg_tulu2/data.csv')
data['pred_graphs'] = ''

## MY VERSION trivial one##
vocab_size = hmm_model.vocab_size
eos_token_id = hmm_model.eos_token_id

#prefix = "The overall probability of alarm set by husband is 77%. For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%. Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas:"
#suffix = '</s>'
#soft_constraint = '' # use empty string for no soft constraint
#prompt = f'<|user|>\n"Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in "var1 -> var2" format, separated by commas.<|endoftext|>"{soft_constraint}:\n{prefix}\n<|assistant|>\n'

prefix = data['prefix'].iloc[1000]
d_prompt = data['prompt'].iloc[1000]
#prefix='output a sequence of "A"s: '
#d_prompt='output one sequence with the letter "A". For example: "AAAA"'
suffix = '</s>'
soft_constraint = '' # use empty string for no soft constraint
prompt = f'<|user|>\n"{d_prompt}<|endoftext|>"{soft_constraint}:\n{prefix}\n<|assistant|>\n'


prefix_ids = tokenizer.encode(prefix)[1:]
suffix_ids = tokenizer.encode(suffix)[1:]
prompt_ids = tokenizer.encode(prompt)

#ac_builder = ctrlg.AhoCorasickBuilder(vocab_size)
#ac_builder = ctrlg.NewAhoCorasickBuilder(vocab_size)
#eos_builder = ctrlg.EOSBuilder(vocab_size, eos_token_id)
trivial = ctrlg.TrivialBuilder(tokenizer,vocab_size, eos_token_id)

dfa_graph = trivial.build()


dfa_model = ctrlg.DFAModel(dfa_graph, vocab_size).to(device)

min_new_tokens = 1
max_new_tokens = 30

In [3]:
dfa = {
    "edges": [
    (0, 2, ctrlg.populate_edge(['X', 'Y'], vocab_size, tokenizer)),
    (0, 1, ctrlg.populate_edge(['V'], vocab_size, tokenizer)),
    (1, 2, ctrlg.populate_edge(['1', '2', '3', '4', '5'], vocab_size, tokenizer)),
    (2, 3, ctrlg.populate_edge(['->'], vocab_size, tokenizer)),
    (3, 4, ctrlg.populate_edge(['V'], vocab_size, tokenizer)),
    (3, 5, ctrlg.populate_edge(['X','Y'], vocab_size, tokenizer)),
    (4, 5, ctrlg.populate_edge(['1', '2', '3', '4', '5'], vocab_size, tokenizer)),
    (5, 0, ctrlg.populate_edge([','], vocab_size, tokenizer)),
    ],
    "initial_state": 0,
    "accept_states": set([5]),
    "current_state": 0,
}

In [27]:
e = dfa.get('edges')
ed = [i[2] for i in e if i[0] == 0]
ed1=ed[0]+ed[1]
ed1
[i for i, j in enumerate(ed1) if j]

[478, 612, 1060]

In [3]:
import numpy as np
a=torch.tensor([ 0.0000e+00, -1.4286e+30, -1.4286e+30, -1.4286e+30, -1.4286e+30],
       device='cuda:0')
a

tensor([ 0.0000e+00, -1.4286e+30, -1.4286e+30, -1.4286e+30, -1.4286e+30],
       device='cuda:0')

In [6]:
tokenizer.decode(2)

'</s>'

Step 3. generate with constraints.

Due to the use of @torch.compile, the first run of the following functions could be significantly slower than the later runs.

In [4]:
dic = {} #the keys are the temperatures and the values are a list of the 10 predicted graphs
temp = [1,10,50,100]

# initialze the constraints logits processor
constraint_logits_processor = ctrlg.NewConstraintLogitsProcessor(
    hmm_model, dfa_model,
    min_new_tokens, max_new_tokens,
    prompt_ids, dfa, prefix_ids=prefix_ids, suffix_ids=suffix_ids)


# set the hmm_batch_size & temperature
beam_size = 32 # sample 128 sequences
temperature = 0.7
constraint_logits_processor.hmm_batch_size = beam_size
constraint_logits_processor.temperature = temperature



# generate with sampling, temperature=0.7
input_ids = torch.tensor([prompt_ids], device=device)
outputs = base_model.generate(
        input_ids=input_ids, do_sample=True,
        num_return_sequences=beam_size, 
        min_new_tokens=min_new_tokens, max_new_tokens=max_new_tokens,
        logits_processor=LogitsProcessorList([constraint_logits_processor]),
        pad_token_id=tokenizer.eos_token_id,
    )


# extract the generated ids; removing prompt ids; remove suffix ids that are (partially) generated
generated_ids = ctrlg.extract_generated_ids(outputs.tolist(), prompt_ids, suffix_ids, eos_token_id)

# filter 75% of the generated ids by how well they connect with the suffix
generated_ids = ctrlg.rank_generated_ids(base_model, generated_ids, prompt_ids, suffix_ids,
                                            suffix_logits_only=True, suffix_length_cap=5)[:32]
# rank the generated ids by the base_model for higher quality
generated_ids = ctrlg.rank_generated_ids(base_model, generated_ids, prompt_ids, suffix_ids)

dic[temperature] = []
# print top 10 outputs
for idx, generated in enumerate(generated_ids[:1]):
    print(f'{idx}. ' + tokenizer.decode(generated, skip_special_tokens=True) + \
          tokenizer.decode(suffix_ids, skip_special_tokens=True))
    #dic[temperature].append(tokenizer.decode(generated, skip_special_tokens=True) + \
    #      tokenizer.decode(suffix_ids, skip_special_tokens=True))
    #print(f'{idx}. ' + tokenizer.decode(prefix_ids, skip_special_tokens=True) + \
    #      ' ' + '\033[1m' + tokenizer.decode(generated, skip_special_tokens=True) + '\033[0m' + ' ' + \
    #      tokenizer.decode(suffix_ids, skip_special_tokens=True))

#data.at[i, 'pred_graphs'] = dic

tensor([[-2.8792e+01, -3.4110e+01,        -inf,  ..., -2.6699e+01,
         -2.3268e+01, -1.4286e+30],
        [-2.7192e+01, -3.2511e+01,        -inf,  ..., -2.5100e+01,
         -2.1668e+01, -1.4286e+30],
        [-2.7192e+01, -3.2511e+01,        -inf,  ..., -2.5100e+01,
         -2.1668e+01, -1.4286e+30],
        ...,
        [-2.7192e+01, -3.2511e+01,        -inf,  ..., -2.5100e+01,
         -2.1668e+01, -1.4286e+30],
        [-2.7192e+01, -3.2511e+01,        -inf,  ..., -2.5100e+01,
         -2.1668e+01, -1.4286e+30],
        [-2.7192e+01, -3.2511e+01,        -inf,  ..., -2.5100e+01,
         -2.1668e+01, -1.4286e+30]], device='cuda:0')
tensor([[-3.4780e+01, -3.8111e+01, -1.2291e+01,  ..., -3.1889e+01,
         -3.0757e+01, -1.4286e+30],
        [-3.4780e+01, -3.8111e+01, -1.2291e+01,  ..., -3.1889e+01,
         -3.0757e+01, -1.4286e+30],
        [-3.4780e+01, -3.8111e+01, -1.2291e+01,  ..., -3.1889e+01,
         -3.0757e+01, -1.4286e+30],
        ...,
        [-3.4780e+01, -3.8111

In [11]:
#tokenizer.encode('3')
#[29892, 29901,     2,   478,   313]
tokenizer.decode([   2, 1919,    3,    1,    0])

'</s> ,\x00<s><unk>'

In [4]:
pd.set_option('display.max_colwidth', None)
data.iloc[1000]

Unnamed: 0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         1000
prompt                                                                                                                                                                                          

In [6]:
a = {}
a['x']=[]
a['x'].append(2)
a['x'].append(3)
a

{'x': [2, 3]}

In [ ]:
df.at['linha', 'col']

In [88]:
import pandas as pd
data = pd.read_json('/home/bazaluk/master_project/cladder.json')
data = data.dropna().reset_index(drop=True)

data = data.assign(step0 = lambda x: (x['reasoning']))
df = data['step0'].apply(lambda x: (x.get("step0")))
data['step0'] = df

data = data.assign(step1 = lambda x: (x['reasoning']))
df = data['step1'].apply(lambda x: (x.get("step1")))
data['step1'] = df

#step2) query type
data = data.assign(step2 = lambda x: (x['meta']))
df = data['step2'].apply(lambda x: (x.get("query_type")))
data['step2'] = df

#step3) formalize query
data = data.assign(step3 = lambda x: (x['reasoning']))
df = data['step3'].apply(lambda x: (x.get("step2")))
data['step3'] = df

#step4) extract all available data
data = data.assign(step4 = lambda x: (x['reasoning']))
df = data['step4'].apply(lambda x: (x.get("step4")))
data['step4'] = df

#step5) deduce estimand
data = data.assign(step5 = lambda x: (x['reasoning']))
df = data['step5'].apply(lambda x: (x.get("step3")))
data['step5'] = df

#step6)Insert the relevant data in Step 4 into the estimand, perform basic arithmetic calculations
data = data.assign(step6 = lambda x: (x['reasoning']))
df = data['step6'].apply(lambda x: (x.get("step5")))
data['step6'] = df

#end) derive the final answer
data = data.assign(end = lambda x: (x['reasoning']))
df = data['end'].apply(lambda x: (x.get("end")))
data['end'] = df

data

,question_id,desc_id,given_info,question,answer,meta,reasoning,step0,step1,step2,step3,step4,step5,step6,end
0,4,alarm-mediation-nde-model0-spec0-q0,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",nde,"E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.08\nP(Y=1 | X=0, V2=1) ...","\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y...",0.74 * (0.86 - 0.41) + 0.24 * (0.54 - 0.08) = ...,0.32 > 0
1,7,alarm-mediation-ate-model1-spec1-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",ate,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.26\nP(Y=1 | X=1) = 0.76,P(Y=1|X=1) - P(Y=1|X=0),0.76 - 0.26 = 0.50,0.50 > 0
2,8,alarm-mediation-marginal-model1-spec1-q0,The overall probability of alarm set by husban...,Is ringing alarm more likely than silent alarm...,yes,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",marginal,P(Y),P(X=1) = 0.77\nP(Y=1 | X=0) = 0.26\nP(Y=1 | X=...,P(Y | X=1)*P(X=1) + P(Y | X=0)*P(X=0),0.77*0.76 - 0.23*0.26 = 0.64,0.64 > 0
3,15,alarm-mediation-ate-model3-spec3-q1,"For husbands that don't set the alarm, the pro...",Will alarm set by husband decrease the chance ...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",ate,E[Y | do(X = 1)] - E[Y | do(X = 0)],P(Y=1 | X=0) = 0.20\nP(Y=1 | X=1) = 0.68,P(Y=1|X=1) - P(Y=1|X=0),0.68 - 0.20 = 0.49,0.49 > 0
4,21,alarm-mediation-nie-model4-spec4-q0,For husbands that don't set the alarm and wive...,Does husband positively affect alarm clock thr...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...,Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",nie,"E[Y_{X=0, V2=1} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.11\nP(Y=1 | X=0, V2=1) ...","\sum_{V2 = v} P(Y=1|X =0,V2 = v)*[P(V2 = v | X...",0.01 * (0.60 - 0.11)+ 0.61 * (0.92 - 0.46)= -0.29,-0.29 < 0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8527,31012,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if not zuph inste...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",det-counterfactual,Y_{X=0} = 1 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 0 or 1,1
8528,31014,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is glimx if zuph instead o...,yes,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",det-counterfactual,Y_{X=1} = 1 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 1 or 1,1
8529,31015,nonsense9-fork-det-counterfactual-model3842-sp...,We know that zuph or jyka causes glimx. We obs...,Would an individual is not glimx if zuph inste...,no,"{'story_id': 'nonsense9', 'graph_id': 'fork', ...",{'step0': 'Let V2 = jyka; X = zuph; Y = glimx....,Let V2 = jyka; X = zuph; Y = glimx.,"X->Y,V2->Y",det-counterfactual,Y_{X=1} = 0 | V2=1,V2 = 1\nY = X or V2,"Solve for Y, given the evidence and the action",Y = 1 = 1 or 1,0
8530,31016,nonsense9-fork-det-counterfactual-model3843-sp...,We know that zuph and jyka causes glimx. We ob...,Would an individual is glimx

In [89]:
#create a list of real variable names and their respective representation e.g. [['X','wife'],['Y','husband']]
def list_representation(x):
    x = x.get('step0')
    x = x[4:len(x)-1] #erase "Let " and the last "."
    x = x.split('; ')

    aux = []
    for i in range(len(x)):
        aux.append(x[i].split(" = "))
    return aux

#receives a list from list_representation and returns the prompt with it
def prompt_repres(x):
    x = x.get('step0')
    x = x[4:len(x)-1] #erase "Let " and the last "."
    x = x.split('; ')

    list_rep = []
    for i in range(len(x)):
        list_rep.append(x[i].split(" = "))
    
    prompt = ""
    for i in range(len(list_rep)):
        prompt += "Use "+list_rep[i][0]+" to represent "+list_rep[i][1]+". "
    return prompt

def create_string(str_list):
    string = ""
    for i in range(str_list):
        string += str_list[i]
    return string

In [90]:
#create columnn with complete prompt to be given to the LLM
paraphrases = [
            "Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships. ",
            "Think of a self-contained, hypothetical setting with just the specified conditions, and devoid of any unknown factors or causal connections. ",
            "Consider a self-contained, hypothetical world with solely the mentioned conditions, and is free from any hidden factors or cause-and-effect relationships. ",
            "Imagine a self-contained, hypothetical setting with merely the stated conditions, and absent any unmentioned factors or causative links. ",
            "Think of a self-contained, hypothetical world with only the given conditions, and is void of any unknown factors or causative relationships. ",
        ]

prompt_end0 = "Extract the causal graph: Identify the causal graph that \
depicts the relationships in the scenario. "
prompt_end1 = 'The diagram should simply consist of edges denoted in "var1 -> var2" format, separated by commas.'
prefix_end1 = 'The diagram should simply consist of edges denoted in "var1 -> var2" format, separated by commas:'
prefix_end0 = " Extract the causal graph: Identify the causal graph that \
depicts the relationships in the scenario. "

data['prompt'] = data.apply(lambda x: (prompt_end0+prompt_repres(x['reasoning'])+prompt_end1),axis=1)
data['prefix'] = data.apply(lambda x: (x['given_info']+prefix_end0+prompt_repres(x['reasoning'])+prefix_end1),axis=1)

#df = data[['prompt', 'formal_form', 'graph', 'query_type']]
#data['prefix'].iloc[0]
#data['prompt'].iloc[0]

#/media/data/bazaluk/ctrlg_tulu2/data.csv
data[['prompt','prefix','step1']].to_csv('/media/data/bazaluk/ctrlg_tulu2/data.csv')
#data['prompt'].iloc[0]
data['prefix'].iloc[1]

'For husbands that don\'t set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%. Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in "var1 -> var2" format, separated by commas:'

In [15]:
import pandas as pd
#######################################################################################################################
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # set your cuda device
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import torch
import ctrlg
from transformers import AutoModelForCausalLM, AutoTokenizer, LogitsProcessorList

device = 'cuda'

# load the pretrained base_model and hmm_model;
BASE_MODEL_PATH = f'ctrlg/tulu2-7b_writing-prompts'
HMM_MODEL_PATH = f'ctrlg/hmm_tulu2-7b_writing-prompts_32768'

base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_PATH).to(device)
base_model.eval()
base_model.half() # fp16 inference
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH)
hmm_model = ctrlg.HMM.from_pretrained(HMM_MODEL_PATH).to(device)

#######################################################################################################################
## MY VERSION ##
vocab_size = hmm_model.vocab_size
eos_token_id = hmm_model.eos_token_id

data = pd.read_csv('/media/data/bazaluk/ctrlg_tulu2/data.csv')
data['pred_graphs'] = ''

for i in range(3):
#for i in range(len(data)):
    vocab_size = hmm_model.vocab_size
    eos_token_id = hmm_model.eos_token_id

    prefix = data['prefix'].iloc[i]
    d_prompt = data['prompt'].iloc[i]
    suffix = '</s>'
    soft_constraint = '' # use empty string for no soft constraint
    prompt = f'<|user|>\n"{d_prompt}<|endoftext|>"{soft_constraint}:\n{prefix}\n<|assistant|>\n'


    prefix_ids = tokenizer.encode(prefix)[1:]
    suffix_ids = tokenizer.encode(suffix)[1:]
    prompt_ids = tokenizer.encode(prompt)

    ac_builder = ctrlg.AhoCorasickBuilder(vocab_size)
    eos_builder = ctrlg.EOSBuilder(vocab_size, eos_token_id)


    dfa_graphs = []
    #keyphrases = [['towering'], ['reach the sky'], ['reflected'], ['lake']]
    keyphrases = [[' X ', ' Y ', ' V2 ', ' V3 ', ' V4 ', ' V5 '],
                    [' ->'],
                     [' X ', ' Y ', ' V2 ', ' V3 ', ' V4 ', ' V5 ']
                 ]

    for keyphrase in keyphrases:
        patterns = [tokenizer.encode(x)[1:] for x in keyphrase]
        dfa_graphs.append(ac_builder.build(patterns))
    dfa_graphs.append(eos_builder.build())


    dfa_graph = ctrlg.DFA_prod(dfa_graphs, mode='intersection')
    dfa_model = ctrlg.DFAModel(dfa_graph, vocab_size).to(device)

    min_new_tokens = 1
    max_new_tokens = 32
    
    ############################################################################
    # initialze the constraints logits processor
    constraint_logits_processor = ctrlg.ConstraintLogitsProcessor(
        hmm_model, dfa_model,
        min_new_tokens, max_new_tokens,
        prompt_ids, prefix_ids=prefix_ids, suffix_ids=suffix_ids)


    # set the hmm_batch_size & temperature
    beam_size = 32 # sample 128 sequences
    temperature = 0.9
    constraint_logits_processor.hmm_batch_size = beam_size
    constraint_logits_processor.temperature = temperature


    # generate with sampling, temperature=0.7
    input_ids = torch.tensor([prompt_ids], device=device)
    outputs = base_model.generate(
            input_ids=input_ids, do_sample=True,
            num_return_sequences=beam_size, 
            min_new_tokens=min_new_tokens, max_new_tokens=max_new_tokens,
            logits_processor=LogitsProcessorList([constraint_logits_processor]),
            pad_token_id=tokenizer.eos_token_id,
        )


    # extract the generated ids; removing prompt ids; remove suffix ids that are (partially) generated
    generated_ids = ctrlg.extract_generated_ids(outputs.tolist(), prompt_ids, suffix_ids, eos_token_id)

    # filter 75% of the generated ids by how well they connect with the suffix
    generated_ids = ctrlg.rank_generated_ids(base_model, generated_ids, prompt_ids, suffix_ids,
                                                suffix_logits_only=True, suffix_length_cap=5)[:32]
    # rank the generated ids by the base_model for higher quality
    generated_ids = ctrlg.rank_generated_ids(base_model, generated_ids, prompt_ids, suffix_ids)

    # save top 10 outputs
    pred_graphs = []
    for idx, generated in enumerate(generated_ids[:10]):
        pred_graphs.append(tokenizer.decode(generated, skip_special_tokens=True) + \
              tokenizer.decode(suffix_ids, skip_special_tokens=True))
    data.at[i, 'pred_graphs'] = pred_graphs

data.to_csv('teste.csv')

/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.read_csv('/media/data/bazaluk/ctrlg_tulu2/graph_pred.csv')

,Unnamed: 0.1,Unnamed: 0,prompt,prefix,step1,pred_graphs
0,0,0,"Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in ""var1 -> var2"" format, separated by commas.","For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%. Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in ""var1 -> var2"" format, separated by commas:","X->V2,X->Y,V2->Y","{1: ['X -> V2', 'X -> V2', 'X -> V2', 'X -> V2', 'X -> V2', 'X -> V2', 'X -> Y', 'X -> Y', 'X -> Y', 'X -> Y'], 10: ['V2 -> Y', 'V2 -> Y', 'V2 -> Y', 'X -> V2', 'V2 -> V1', 'V2 -> X', 'V2 -> X', 'X -> Y', 'X -> Y', 'X -> Y'], 50: ['X -> V2', 'V2 -> X', 'V2 -> X', 'Y -> V2', 'X -> Y', 'X -> Y', 'V1 -> X', 'Y -> X , X -> Y', 'Y -> X', 'Y -> X'], 100: ['X -> V2', 'V3 -> V4 , V2 -> V2 , V4 -> V2 , V5 -> V3', 'X -> Y', 'X -> Y', 'V1 -> Y', 'V1 -> Y', 'V1 -> Y', 'X -> V1', 'Y -> X', 'Y -> X']}"
1,1,1,"Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in ""var1 -> var2"" format, separated by commas.","For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%. Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in ""var1 -> var2"" format, separated by commas:","X->V2,X->Y,V2->Y","{1: ['X -> V2', 'X -> V2', 'X -> V2', 'X -> V2', 'X -> Y', 'X -> Y', 'X -> Y', 'X -> Y', 'X -> Y', 'X -> Y'], 10: ['V2 -> Y', 'V2 -> Y', 'X -> V2', 'X -> V2', 'V2 -> X', 'V2 -> X', 'V1 -> Y', 'V1 -> X', 'X -> Y', 'X -> Y'], 50: ['X -> V2', 'V2 -> X', 'V2 -> X , X -> Y', 'V1 -> X', 'V1 -> X', 'X -> Y', 'X -> Y', 'X -> Y', 'X -> Y', 'X -> V1'], 100: ['X -> V2', 'V1 -> Y', 'Y -> V2', 'V2 -> V3', 'X -> Y', 'X -> Y', 'X -> Y', 'X -> Y', 'V2 -> X , V3 -> X', 'V1 -> V2 , Y -> X , X -> X']}"
2,2,2,"Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in ""var1 -> var2"" format, separated by commas.","The overall probability of alarm set by husband is 77%. For husbands that don't set the alarm, the probability of ringing alarm is 26%. For husbands that set the alarm, the probability of ringing alarm is 76%. Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in ""var1 -> var2"" format, separated by commas:","X->V2,X->Y,V2->Y","{1: ['X -> V2', 'X -> V2', 'X -> V2', 'X -> V2', 'X -> V2', 'X -> Y', 'X -> Y', 'X -> Y', 'X -> Y', 'X -> Y'], 10: ['V2 -> Y', 'V1 -> V2', 'V2 -> X', 'X -> V2', 'X -> V2', 'X -> V2', 'Y -> V2',

In [6]:
#[(k[0], k[1], v) for k, v in trans.items() if k[0] != tuple() or k[1] != tuple()]
a=[1,2,3]
b=[4,5,6]
[(a[i],b[i]) for i in range(3) if a[i] != 2 and b[i] != 6]

[(1, 4)]

In [10]:
a= {
  "Germany": "Berlin", 
  "Canada": "Ottawa", 
  "England": "London"
}
list(a.items())[0]

('Germany', 'Berlin')

In [13]:
pd.read_csv('/media/data/bazaluk/cladder_gpt3.5.csv')

,Unnamed: 0,question_id,desc_id,given_info,question,answer,meta,reasoning,step0,step1,...,step3,step4,step5,step6,end,prompt,predicted graph,predicted query type,predicted final answer,predicted text
0,0,4,alarm-mediation-nde-model0-spec0-q0,"For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%.","If we disregard the mediation effect through wife, would husband positively affect alarm clock?",yes,"{'story_id': 'alarm', 'graph_id': 'mediation', 'mediators': ['V2'], 'polarity': True, 'groundtruth': 0.322381664440195, 'query_type': 'nde', 'rung': 3, 'formal_form': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]', 'given_info': {'p(Y | X, V2)': [[0.08430222457648501, 0.539461052145868], [0.406150970112692, 0.8620283206949241]], 'p(V2 | X)': [0.741686618881911, 0.23519324071521203]}, 'estimand': '\\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y=1|X=0, V2=v)]', 'treatment': 'X', 'outcome': 'Y', 'model_id': 0}","{'step0': 'Let X = husband; V2 = wife; Y = alarm clock.', 'step1': 'X->V2,X->Y,V2->Y', 'step2': 'E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]', 'step3': '\\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y=1|X=0, V2=v)]', 'step4': 'P(Y=1 | X=0, V2=0) = 0.08\nP(Y=1 | X=0, V2=1) = 0.54\nP(Y=1 | X=1, V2=0) = 0.41\nP(Y=1 | X=1, V2=1) = 0.86\nP(V2=1 | X=0) = 0.74\nP(V2=1 | X=1) = 0.24', 'step5': '0.74 * (0.86 - 0.41) + 0.24 * (0.54 - 0.08) = 0.32', 'end': '0.32 > 0'}",Let X = husband; V2 = wife; Y = alarm clock.,"X->V2,X->Y,V2->Y",...,"E[Y_{X=1, V2=0} - Y_{X=0, V2=0}]","P(Y=1 | X=0, V2=0) = 0.08\nP(Y=1 | X=0, V2=1) = 0.54\nP(Y=1 | X=1, V2=0) = 0.41\nP(Y=1 | X=1, V2=1) = 0.86\nP(V2=1 | X=0) = 0.74\nP(V2=1 | X=1) = 0.24","\sum_{V2=v} P(V2=v|X=0)*[P(Y=1|X=1,V2=v) - P(Y=1|X=0, V2=v)]",0.74 * (0.86 - 0.41) + 0.24 * (0.54 - 0.08) = 0.32,0.32 > 0,"Imagine a self-contained, hypothetical world with only the following conditions, and without any unmentioned factors or causal relationships. Question: For husbands that don't set the alarm and wives that don't set the alarm, the probability of ringing alarm is 8%. For husbands that don't set the alarm and wives that set the alarm, the probability of ringing alarm is 54%. For husbands that set the alarm and wives that don't set the alarm, the probability of ringing alarm is 41%. For husbands that set the alarm and wives that set the alarm, the probability of ringing alarm is 86%. For husbands that don't set the alarm, the probability of alarm set by wife is 74%. For husbands that set the alarm, the probability of alarm set by wife is 24%. Guidance: Address the question by following the steps below:\nStep 1) Extract the causal graph: Identify the causal graph thatdepicts the relationships in the scenario. Use X to represent husband. Use V2 to represent wife. Use Y to represent alarm clock. The diagram should simply consist of edges denoted in 'var1 -> var2' format, separated by commas. Step 2) Determine the query type: Identify the type of query implied by the main question. Choices include 'marginal probability', 'conditional probability', 'explainingaway effect', 'backdoor adjustment set', 'average treatment effect', 'collider bias', 'normal counterfactual question', 'average treatmenteffect on treated', 'natural direct effect' or 'natural indirect effect'. Your answer should only be a term from the list above, enclosed in quotation marks. Step 3) Extract all the available data. Your answer should contain nothing but marginal probabilities and conditional probabilities in the form 'P(...)=..